<img src="https://theaiengineer.dev/tae_logo_gw_flatter.png" width=35% align=right>

# AI Agents & Automation — Chapter 3
## The Engineering Mindset

&copy; Dr. Yves J. Hilpisch<br>
AI-Powered by GPT-5.

### Overview

This notebook accompanies Chapter 3 — Engineering Mindset. It is self-contained and demonstrates the core ideas with small, readable code cells. Run cells from top to bottom; each code cell is preceded by a short explanation of what it does.


We add small seatbelts: timeouts and retries around a tiny tool to keep the loop responsive and observable.

In [ ]:
import threading  # import threading for concurrency
import time  # import time utilities


def call_with_timeout(fn, *, timeout_ms: int, **kwargs):
    """Run fn in a thread with a timeout.

    Returns a tuple (ok, value_or_error).
    """
    result = {}  # hold the eventual return value
    error = {}  # capture exception text if raised

    def run() -> None:
        try:
            result['value'] = fn(**kwargs)  # store successful result
        except Exception as exc:  # noqa: BLE001
            error['message'] = str(exc)  # record failure text

    worker = threading.Thread(target=run, daemon=True)  # create worker thread
    worker.start()  # launch worker
    worker.join(timeout_ms / 1000)  # wait up to timeout_ms
    if worker.is_alive():
        return False, 'timeout'  # signal timeout
    if error:
        return False, error['message']  # propagate failure
    return True, result.get('value')  # success and payload


def flaky() -> str:
    time.sleep(0.2)  # simulate slow work
    return 'ok'  # deterministic response


def demo() -> None:
    ok, value = call_with_timeout(flaky, timeout_ms=100)  # guarded call
    print({'ok': ok, 'out': value})  # display outcome


demo()  # run demo


Try a retry loop with backoff; print `latency_ms` and `status` per attempt.

<img src="https://theaiengineer.dev/tae_logo_gw_flatter.png" width=35% align=right>